In [ ]:

# ✅ Check your environment
import torch
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())


In [ ]:
import os
import shutil
import random

# Source folders
SOURCE_IMG_DIR = '/kaggle/input/stationary/train/images'
SOURCE_LBL_DIR = '/kaggle/input/stationary/train/labels'

# Destination base
DEST_BASE = '/kaggle/working/dataset'

# Create folder structure
for folder in ['images/train', 'images/valid', 'labels/train', 'labels/valid']:
    os.makedirs(os.path.join(DEST_BASE, folder), exist_ok=True)

# Get all image filenames
image_files = [f for f in os.listdir(SOURCE_IMG_DIR) if f.endswith('.jpg')]
random.shuffle(image_files)

# Split 80-20
split_idx = int(0.8 * len(image_files))
train_files = image_files[:split_idx]
valid_files = image_files[split_idx:]

def move_files(files, split):
    for img_file in files:
        label_file = img_file.replace('.jpg', '.txt')
        
        # Copy image
        shutil.copy(os.path.join(SOURCE_IMG_DIR, img_file),
                    os.path.join(DEST_BASE, f'images/{split}', img_file))
        
        # Copy label
        shutil.copy(os.path.join(SOURCE_LBL_DIR, label_file),
                    os.path.join(DEST_BASE, f'labels/{split}', label_file))

move_files(train_files, 'train')
move_files(valid_files, 'valid')

print(f"✅ Done! Train: {len(train_files)} | Valid: {len(valid_files)}")


In [ ]:
yaml_content = """
train: /kaggle/working/dataset/images/train
val: /kaggle/working/dataset/images/valid

nc: 4
names: ['sharpener', 'pencil', 'scale', 'eraser']
"""

with open('/kaggle/working/dataset/data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml created successfully!")


In [ ]:
!cat /kaggle/working/dataset/data.yaml


In [ ]:
# Install YOLOv5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt



In [ ]:

# Train using our new split
!python train.py --img 640 --batch 16 --epochs 50 --data /kaggle/working/dataset/data.yaml --weights yolov5s.pt --name stationary-detector

# Save model
!cp runs/train/stationary-detector/weights/best.pt /kaggle/working/

In [ ]:
import streamlit as st
import torch
from PIL import Image
import numpy as np
import cv2

# 🚀 Load model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/best.pt', force_reload=True)

# 🏷️ Prices for each item
prices = {
    'sharpener': 20,
    'pencil': 10,
    'scale': 30,
    'eraser': 15
}

st.title('🖍️ Stationary Item Detector & Bill Calculator')

uploaded_file = st.file_uploader("📸 Upload an image of stationery items", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # 🔍 Inference
    results = model(image)
    labels, cords = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
    img_np = np.array(image)

    for *box, conf, cls in results.xyxy[0]:
        x1, y1, x2, y2 = map(int, box)
        label = model.names[int(cls)]
        cv2.rectangle(img_np, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img_np, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    st.image(img_np, caption='Detected Items', use_column_width=True)

    # 🧾 Bill calculation
    detected_items = [model.names[int(cls)] for cls in labels]
    bill = 0
    st.subheader('🧾 Itemized Bill:')
    for item in set(detected_items):
        qty = detected_items.count(item)
        st.write(f"{item.capitalize()} × {qty} = {prices[item]*qty} PKR")
        bill += prices[item] * qty

    st.success(f"💰 Total: {bill} PKR")
